# LSH Cascade 検索システム - 最終レポート

## 概要

本プロジェクトでは、大規模ベクトル検索の高速化を目的として、**ITQ-LSH（Iterative Quantization Locality Sensitive Hashing）** と **Overlapセグメントフィルタリング** を組み合わせたカスケード検索システムを開発・評価しました。

### 最終推奨パラメータ

```
埋め込みモデル: intfloat/multilingual-e5-base (768次元)
ITQビット数: 128 bits
Overlap設定: width=8, stride=4 (31セグメント)
```

### 性能サマリー（40万件Wikipedia）

| 指標 | 値 |
|------|----|
| 候補削減率 | 87.5% |
| Recall@10 (limit=1000) | 83.7% |
| ベースラインRecall | 88.8% |
| Recall低下 | -5.1pt |

---

## 実験ノートブック一覧

### Phase 1: モデル・ビット数の評価 (44-46)

| No. | ノートブック | 目的 | 主要な発見 |
|-----|-------------|------|------------|
| 44 | [44_e5_base_bits_evaluation.ipynb](44_e5_base_bits_evaluation.ipynb) | E5-baseでのITQビット数評価 | 64 bitsで99.6% Recall@10達成、ストレージ効率重視なら64 bits推奨 |
| 45 | [45_e5_base_fastembed_compatibility.ipynb](45_e5_base_fastembed_compatibility.ipynb) | FastEmbed互換性検証 | FastEmbedはE5-base未対応、モデル変更時はITQ再学習必要 |
| 46 | [46_fastembed_practical_speed.ipynb](46_fastembed_practical_speed.ipynb) | FastEmbed実用速度評価 | カスタム登録でE5シリーズ使用可能、CPU推論は遅い |

### Phase 2: Overlap評価・最適化 (47-49)

| No. | ノートブック | 目的 | 主要な発見 |
|-----|-------------|------|------------|
| 47 | [47_fastembed_e5base_optimization.ipynb](47_fastembed_e5base_optimization.ipynb) | FastEmbed最適化 + Overlap小規模評価 | 5000件ではOverlapフィルタ効果なし（全件ヒット） |
| 48 | [48_wikipedia_400k_embedding.ipynb](48_wikipedia_400k_embedding.ipynb) | Wikipedia 40万件データセット構築 | E5-base埋め込み + ITQ(64/96/128 bits)学習・永続化 |
| 49 | [49_overlap_evaluation_400k.ipynb](49_overlap_evaluation_400k.ipynb) | 40万件規模Overlap評価 | **128b + Overlap(8,4)** が最適バランス |

---

## 主要な技術的発見

### 1. ITQビット数の選択

| ビット数 | Recall@10 (40万件) | ストレージ | 推奨用途 |
|----------|-------------------|-----------|----------|
| 64 bits | 73.8% | 8 bytes | ストレージ重視 |
| 96 bits | 82.5% | 12 bytes | バランス |
| **128 bits** | **88.8%** | 16 bytes | **精度重視（推奨）** |

**結論**: 128 bitsが最も高いRecallを達成。ストレージ増加（+4-8 bytes/doc）は許容範囲。

### 2. Overlapセグメント設定

128 bitsでの各設定の比較:

| 設定 | セグメント数 | 削減率 | Recall@1000 | 評価 |
|------|-------------|--------|-------------|------|
| **(8, 4)** | **31** | **87.5%** | **83.7%** | **推奨** |
| (8, 2) | 61 | 82.2% | 86.0% | 高Recall |
| (12, 6) | 15 | 99.2% | 44.6% | 非推奨 |
| (16, 8) | 11 | 99.9% | 21.1% | 非推奨 |

**結論**: 
- **Overlap(8, 4)** が削減率とRecallの最適バランス
- セグメント幅12bit以上は削減率が高すぎてRecall大幅低下

### 3. E5-base vs E5-large

非公開データでの汎化性能比較:

| 指標 | E5-large (04) | E5-base (05) | 改善 |
|------|--------------|--------------|------|
| 外部クエリ Step1 R@10 | 62.0% | **82.0%** | **+20.0pt** |
| 外部クエリ 最終R@10 | 62.0% | **79.5%** | **+17.5pt** |
| 埋め込み次元 | 1024 | 768 | -25% |
| ITQストレージ | 同じ | 同じ | - |

**結論**: E5-baseの方が外部クエリに対する汎化性能が高い。次元数も小さく効率的。

### 4. データ規模の影響

| データ規模 | Overlapフィルタ効果 | 推奨手法 |
|-----------|---------------------|----------|
| ~5,000件 | 効果なし（全件ヒット） | 2段階検索 |
| ~3,000件 | 一部効果あり | 2段階検索 |
| **40万件以上** | **効果大（87%削減）** | **3段階カスケード** |

**結論**: Overlapフィルタは大規模データ（数十万件以上）で真価を発揮。小規模データでは2段階検索（全件ハミング距離→コサイン）が実用的。

---

## 永続化ファイル一覧

### ITQモデル・ハッシュ（data/） (データは別途)

```
data/
├── itq_e5_base_64bits.pkl          # ITQモデル (64 bits)
├── itq_e5_base_96bits.pkl          # ITQモデル (96 bits)
├── itq_e5_base_128bits.pkl         # ITQモデル (128 bits) ★推奨
├── wikipedia_400k_e5_base.duckdb   # 埋め込み + HNSWインデックス (3.4GB)
├── wikipedia_400k_e5_base_embeddings.npy  # 埋め込みベクトル (1.2GB)
├── wikipedia_400k_e5_base_meta.npz        # メタデータ
├── wikipedia_400k_e5_base_hashes_64bits.npy   # ハッシュ (25MB)
├── wikipedia_400k_e5_base_hashes_96bits.npy   # ハッシュ (37MB)
└── wikipedia_400k_e5_base_hashes_128bits.npy  # ハッシュ (49MB) ★推奨
```

### 使用方法

```python
from src.itq_lsh import ITQLSH

# ITQモデルロード
itq = ITQLSH.load('data/itq_e5_base_128bits.pkl')

# 埋め込みからハッシュ生成
hashes = itq.transform(embeddings)  # (n_samples, 128)
```

---

## 3段階カスケード検索アーキテクチャ

```
┌─────────────────────────────────────────────────────────────┐
│                      Query Embedding                        │
│                    (E5-base, 768次元)                       │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Step 1: Overlap Segment Filter (OR条件)                    │
│  - 128 bits → 31セグメント (8bit幅, 4bitストライド)         │
│  - いずれかのセグメントが一致する文書を候補に               │
│  - 削減率: 87.5% (399k → 50k)                               │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Step 2: Hamming Distance Sort                              │
│  - 候補文書のハミング距離を計算                             │
│  - 上位N件に絞り込み (例: 1000件)                           │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Step 3: Cosine Similarity Ranking                          │
│  - 絞り込んだ候補でコサイン類似度を計算                     │
│  - Top-K (例: 10件) を返却                                  │
└─────────────────────────────────────────────────────────────┘
```

---

## 計算速度評価

これまでの実験ノートブックから計測された処理時間をまとめます。

### 1. 事前処理（オフライン・1回のみ）

| 処理 | 時間 | 備考 | 計測元 |
|------|------|------|--------|
| ITQ学習 (64 bits) | 11.0秒 | 40万件での学習 | [48](48_wikipedia_400k_embedding.ipynb) |
| ITQ学習 (96 bits) | 15.9秒 | 〃 | [48](48_wikipedia_400k_embedding.ipynb) |
| ITQ学習 (128 bits) | 21.6秒 | 〃 | [49](49_overlap_evaluation_400k.ipynb) |
| ITQハッシュ計算（transform） | 0.4秒 | 40万件全体、約1μs/件 | [48](48_wikipedia_400k_embedding.ipynb) |

→ 事前処理は軽量。40万件でも20秒程度で完了。

### 2. 検索時処理の比較（40万件、内部クエリ100件平均）

| 手法 | 候補数 | 処理時間 | Recall@10 | 評価 | 計測元 |
|------|--------|----------|-----------|------|--------|
| 2段階検索 | 2,000件 | 35.5ms | 91.3% | **推奨** | [30](30_multistage_lsh_filtering.ipynb) |
| 2段階検索 | 5,000件 | 39.3ms | 96.0% | 高精度 | [30](30_multistage_lsh_filtering.ipynb) |
| 多段階(16seg/5000/500) | 500件 | 14.5ms | 72.3% | 低Recall | [30](30_multistage_lsh_filtering.ipynb) |
| HNSW (DuckDB) | - | 56.9ms | 96.6% | 高精度だが低速 | [30](30_multistage_lsh_filtering.ipynb) |

### 3. 手法別の特徴

**2段階検索（ITQ LSH → コサイン類似度）**
- Step1: 全件ハミング距離計算（ビット演算で高速）
- Step2: 候補に対してコサイン類似度計算
- 候補2,000件で91%、5,000件で96%のRecall達成

**多段階検索（セグメントOR → ハミング → コサイン）**
- Step1でのセグメント完全一致が厳しすぎて良い候補を逃す
- 処理は高速だがRecallが低下

**HNSW (DuckDB VSS拡張)**
- 最高精度（96.6%）だがDuckDB経由のオーバーヘッドあり
- 専用ライブラリ（FAISS, hnswlib）ではより高速化の可能性

### 4. 速度に関する結論

- **2段階検索が最適バランス**: 35ms程度で91%+ Recall
- **HNSWより約40%高速**: DuckDB HNSW（57ms）vs 2段階（35ms）
- **ITQ事前処理は軽量**: 40万件で約20秒、インデックス構築は1回のみ
- **スケーラビリティ**: ハミング距離計算はビット演算で高速、データ規模に対して線形

---

## 追加実験: Firestore/RDBMSでのハミング距離検索手法（実験51-56）

### 背景と目的

FirestoreやRDBMSにはハミング距離を直接計算する機能がないため、128ビットハッシュに対して効率的にハミング距離が近いものを見つける手法を検証しました。

### 実験一覧

| No. | ノートブック | 目的 | 主要な発見 |
|-----|-------------|------|------------|
| 51 | [51_popcount_filtering.ipynb](51_popcount_filtering.ipynb) | ポップカウント（Hamming Weight）フィルタ | 三角不等式による削減は限定的 |
| 52 | [52_pivot_based_indexing.ipynb](52_pivot_based_indexing.ipynb) | ピボットベースインデックス | **8ピボットで高効率なフィルタリング実現** |
| 53 | [53_gray_code_linearization.ipynb](53_gray_code_linearization.ipynb) | グレイコード空間充填曲線 | B-Tree範囲検索での効果は限定的 |
| 54 | [54_combined_filtering_evaluation.ipynb](54_combined_filtering_evaluation.ipynb) | 手法組み合わせ評価 | ピボット単独が最もシンプルで効果的 |
| 55 | [55_overlap_vs_pivot.ipynb](55_overlap_vs_pivot.ipynb) | Overlap vs ピボット詳細比較 | ピボットがストレージ効率・DB実装で優位 |
| 56 | [56_small_data_pivot_evaluation.ipynb](56_small_data_pivot_evaluation.ipynb) | データ規模別評価 | **データ特性により最適戦略が異なる** |

### 5. ピボットベースフィルタリング（実験52, 55）

Vantage Point Treeの考え方をRDBMS/Firestoreに適用した手法です。

**原理**:
- 固定の参照点（ピボット）を8個選択し、各文書とのハミング距離を事前計算
- 三角不等式により `|d(q, pivot) - d(doc, pivot)| ≤ d(q, doc)` が成立
- ピボット距離の差がthreshold以内の文書のみを候補とする

**40万件Wikipediaでの性能比較**:

| 戦略 | 保持フィールド数 | 削減率 | Filter Recall | Recall@10 | DB実装 |
|------|-----------------|--------|---------------|-----------|--------|
| Overlap (8,4) | 31 | 87.7% | 82.4% | 73.8% | 困難（OR条件31個） |
| **Pivot t=15** | **8** | **73.9%** | **85.9%** | **74.3%** | **容易（範囲検索）** |
| Pivot t=20 | 8 | 46.2% | 89.8% | 75.5% | 容易 |

**Firestore実装例**:
```javascript
// ピボット戦略: 1ピボットで範囲検索 + クライアント側フィルタ
db.collection('docs')
  .where('pivot_dist_0', '>=', queryDist - 15)
  .where('pivot_dist_0', '<=', queryDist + 15)
  .get()
```

**結論**: 
- ピボット戦略は**ストレージ効率4倍**（8フィールド vs 31フィールド）
- Firestore/RDBMSでの**実装が大幅に容易**（単純な範囲検索）
- Recall@10はOverlapと同等以上

### 6. データ特性と戦略選択（実験56）

**重要な発見**: 削減率とRecallはデータの分布特性に強く依存する

#### 公開データ（Wikipedia）での検証結果

| データ規模 | Overlap削減率 | Pivot t=15削減率 | Pivot t=20削減率 |
|-----------|--------------|-----------------|-----------------|
| 3,000件 | 87.9% | 73.1% | 46.0% |
| 10,000件 | 87.7% | 72.8% | 46.1% |
| 100,000件 | 87.8% | 72.5% | 45.2% |
| 399,029件 | 87.7% | 73.9% | 46.2% |

→ **削減率はデータ規模に依存せず安定**（公開データの場合）

#### ドメイン特化データでの異なる結果

非公開データ（約2,800件）では:
- Overlap (8,4): 45-56%削減（Wikipediaの87%より大幅低下）
- Pivot t=20: 3-5%削減（ほぼフィルタリング効果なし）

**原因の仮説**:
1. ドメイン特化データはハッシュ空間で密集分布
2. 類似文書が多いほどピボット距離・セグメント値が近接
3. Wikipediaは多様なトピックでハッシュ空間に広く分散

#### Recall@10のスケール依存性

| 戦略 | 3,000件 | 399,029件 | 変化 |
|------|---------|-----------|------|
| Overlap (8,4) | 63.0% | 73.8% | +10.8pt |
| Pivot t=15 | 77.2% | 74.3% | -2.9pt |
| Pivot t=20 | 87.9% | 75.5% | **-12.4pt** |

→ **Pivot t=20は小規模では高Recallだが、大規模ではRecall低下**

#### 戦略選択ガイドライン

| データ特性 | 推奨戦略 | 理由 |
|-----------|---------|------|
| 大規模・多様（10万件+） | Pivot t=15〜20 | 安定した削減率とRecall |
| 中規模・汎用（1万〜10万件） | Pivot t=15 | スケール耐性が高い |
| 小規模・ドメイン特化（〜5,000件） | **要事前検証** | データ分布で効果が大きく変動 |

**最終推奨**:
1. **本番適用前に小規模サンプルで削減率を検証**する
2. **Pivot t=15が最もスケール耐性が高い**（削減率73%、Recall 74-78%で安定）
3. ドメイン特化データではOverlap戦略も検討価値あり

### 追加された永続化ファイル（実験51-56）

```
data/
├── pivots_8_furthest_first.npy           # 8ピボット（furthest first選択）
├── wikipedia_400k_pivot_distances_8.npy  # 各文書のピボット距離 (400k × 8)
└── 5X_*.png                              # 実験51-56の可視化グラフ
```

**ピボット使用方法**:
```python
import numpy as np
from src.itq_lsh import hamming_distance

# ピボットとピボット距離をロード
pivots = np.load('data/pivots_8_furthest_first.npy')  # (8, 128)
pivot_distances = np.load('data/wikipedia_400k_pivot_distances_8.npy')  # (400k, 8)

# クエリのピボット距離を計算
query_pivot_dists = [hamming_distance(query_hash, p) for p in pivots]

# フィルタリング（threshold=15の例）
threshold = 15
mask = np.all(
    np.abs(pivot_distances - query_pivot_dists) <= threshold, 
    axis=1
)
candidates = np.where(mask)[0]
```

---

## 参考文献

1. **ITQ (Iterative Quantization)**
   - Gong et al., "Iterative Quantization: A Procrustean Approach to Learning Binary Codes", CVPR 2011

2. **E5 Embedding Models**
   - Wang et al., "Text Embeddings by Weakly-Supervised Contrastive Pre-training", 2022
   - [intfloat/multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base)

3. **LSH (Locality Sensitive Hashing)**
   - Indyk & Motwani, "Approximate Nearest Neighbors: Towards Removing the Curse of Dimensionality", STOC 1998